In [ ]:
#pip install -q groq wikipedia-api elevenlabs pydub
#apt-get install -y ffmpeg

In [1]:
# !pip install -q groq wikipedia-api elevenlabs pydub
# !apt-get install -y ffmpeg

import wikipediaapi
from groq import Groq
from elevenlabs.client import ElevenLabs
from pydub import AudioSegment
from google.colab import userdata
import io

# --- CONFIGURATION (Use Colab Secrets for real security) ---
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
ELEVEN_API_KEY = userdata.get('ELEVEN_API_KEY')

# Voice IDs
VOICE_RAJ = "hczKB0VbXLcBTn17ShYS"   # George (Male) j57KDF72L6gxbLk4sOo5
VOICE_MEENA = "QbPozfpTbDrP62pzQpyg" # Rachel (Female)

client_groq = Groq(api_key=GROQ_API_KEY)
client_eleven = ElevenLabs(api_key=ELEVEN_API_KEY)

def fetch_wiki(topic):
    wiki = wikipediaapi.Wikipedia(user_agent="ManagerTask/1.0", language='en')
    page = wiki.page(topic)
    return page.summary[:3000] if page.exists() else "Cricket team info."

def generate_script(context):
    print("Generating script...")
    chat_completion = client_groq.chat.completions.create(
        messages=[
            {"role": "system", "content": "Write a 2-min Hinglish convo between Raj and Meena. Raj is a sports fan, Meena is curious. Mix Hindi/English. Format: Raj: [text] and Meena: [text]."},
            {"role": "user", "content": f"Topic: {context}"}
        ],
        model="llama-3.3-70b-versatile",
    )
    return chat_completion.choices[0].message.content

def generate_multi_voice_audio(full_script):
    combined_audio = AudioSegment.empty()
    lines = full_script.strip().split('\n')

    for line in lines:
        if ":" not in line: continue
        
        # Split speaker and text
        speaker, text = line.split(":", 1)
        speaker = speaker.strip()
        text = text.strip()
        
        # Assign Voice
        current_voice = VOICE_RAJ if speaker == "Raj" else VOICE_MEENA
        
        print(f"Synthesizing {speaker}'s line...")
        audio_stream = client_eleven.text_to_speech.convert(
            voice_id=current_voice,
            model_id="eleven_multilingual_v2",
            text=text
        )
        
        # Proper Byte-to-Audio conversion
        audio_data = b"".join(audio_stream)
        segment = AudioSegment.from_file(io.BytesIO(audio_data), format="mp3")
        
        # Add a 500ms pause for natural breathing room
        combined_audio += segment + AudioSegment.silent(duration=500)

    combined_audio.export("final_multi_voice_podcast.mp3", format="mp3")
    print("✅ Success! Your multi-voice podcast is ready.")

# --- RUN ---
try:
    context = fetch_wiki("Mumbai Indians")
    script = generate_script(context)
    generate_multi_voice_audio(script)
except Exception as e:
    print(f"Error: {e}")

ModuleNotFoundError: No module named 'google.colab'